In [1]:
#mounting the drive content that contains the necessary files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Retrieving the pretrained Glove word embeddings which are pretrained on:
#Wikipedia 2014 + Gigaword 5 (6B tokens, 400K vocab, 100d)
#Source: https://nlp.stanford.edu/projects/glove/

!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2022-12-02 09:31:26--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-12-02 09:31:26--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.02MB/s    in 2m 38s  

2022-12-02 09:34:05 (5.19 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflatin

In [3]:
#importing the necessary modules
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import string
from nltk.corpus import stopwords
import numpy as np
import re
import random
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
#obtaining the list of stop words in english
stop_words = set(stopwords.words('english'))
  
#storing the word embeddings as a dictionary
embeddings={}
f_glove=open('/content/glove.6B.100d.txt', 'r',encoding="utf8")

for line in f_glove:
    entries=line.split()
    wordvector = np.asarray(entries[1:], "float32")
    embeddings[entries[0]]=wordvector
f_glove.close()

In [5]:
#function that finds the similar words to the input in the Glove Vector Space that satisfies the POS compatibility
def similars(embeddings,word,ndesired):
    similars=[]
    #obtaining the word's POS tag
    wordtag=nltk.pos_tag([word])
    keys=list(embeddings.keys())
    val=list(embeddings.values())
    #calculating the cosine similarity
    cos=(np.dot(val,embeddings[word])/np.linalg.norm(val,axis=1))/np.linalg.norm(embeddings[word])
    s=np.array([np.array(keys),cos],dtype=object)
    s=np.transpose(s)
    #reversing the list so that the most similar words are on top
    s=s[np.argsort(s[:,1],axis=0),:]
    #filtering the words that satisfy the compatibility constraint
    for i in range(1,len(s)):
        index=len(s)-i-1
        if nltk.pos_tag([s[index][0]])[0][1]==wordtag[0][1]:
            similars.append(s[index][0])
        if len(similars)==ndesired:
            break
    return similars 

In [6]:
#function that cleans the lines from undesired characters as tags,punctuation signs etc.
def cleanline(line):
    tagpattern=r'(<)(.+?)(>)' 
    numberpattern=r'([0-9]+)'
    cl=re.sub(tagpattern,'',line.lower()).replace('\n','').replace('“','').replace('”','').replace('…','').replace('—',' ').replace('‘','').replace('’','').replace('\\' , ' ').replace('/',' ')
    cl=re.sub(numberpattern,' ',cl)
    for char in cl:
            if (char in string.punctuation):
                cl=cl.replace(char,' ')
    return cl

In [7]:
#path of the file that contains the maxims to be augmented
TRAINSETPATHX='/content/drive/MyDrive/Maximsamples.txt'

f=open(TRAINSETPATHX,'r',encoding='utf8')
maximbits=f.readlines()
f.close()
#removing the endline characters
maximbits=[maxim[:-1] for maxim in maximbits]

print("The number of maxims: {}".format(len(maximbits)))

The number of maxims: 10


In [8]:
maximbits

['Lastly the rules which must make up the reference system should be identified according to objective criteria in particular to enable judicial review of the assessments on which that identification is based. It is for the Commission to take into account any factors put forward by the Member State concerned and more generally to carry out its examination in a rigorous and sufficiently reasoned manner in order to enable full judicial review.',
 'Second it is for the Commission when it approves a general aid scheme to take the necessary measures to ensure that the sectoral rules other than the general competition rules will be complied with by the Member State concerned.',
 'Having regard to the foregoing considerations the answer to be given to the first question must be that national measures which provide for a rebate of energy taxes on natural gas and electricity only in the case of undertakings whose activity is shown to consist primarily in the manufacture of goods must be regarde

In [9]:
#breaking the maxim lines into sentences
sentences=[]
nsentence=[] #storing the number of sentences each maxim contains
for line in maximbits:
    count=0
    #splitting sentences using ". " characters
    #listsent=line.replace('\\\n','').split('. ')
    listsent=line.split('. ')
    for i in range(0,len(listsent)):
        sentence=listsent[i]
        #sentences that are too short are eliminated
        if len(sentence)<5:
            continue
        elif (i==len(listsent)-1):
            #the last one is added as it is since we used the character sequence ". " for the seperation
            #and the last sentence contains only "."
            sentences.append(sentence)
            count=count+1
            continue
        else:
            #the "." characters are restored
            if(sentence[-1]!='.'):
                sentence=sentence+'.'
            sentences.append(sentence)
            count=count+1
    nsentence.append(count)

In [10]:
print(len(sentences))
print(len(nsentence))

14
10


In [11]:
perfix=[0.6] #percentage desired
noisesent6=[] 
#iterating over the sentences
for sent in sentences:
    print(sentences.index(sent))
    #tokinizing and capitalizing the sentences
    tokens = nltk.word_tokenize(cleanline(sent).capitalize())
    #lower-casing the tokens
    tokens=[token.lower() for token in tokens]
    #eliminating the stop words
    nonstoptoken = [word for word in tokens if not word in stop_words] 
    tags = nltk.pos_tag(nonstoptoken)
    #collecting the tokens with POS tag in categories nouns, adjectives and adverbs
    tochange=[tag for tag in tags if tag[1][0] in ['J','N','R']]
    nchange=[round(per*len(tochange)) for per in perfix] #number of required replacements due to the percentage given
    #print("Len to change: {}".format(len(tochange)))
    #print("Nchange: {}".format(nchange[0]))
    linemodtotal=sent.lower()
    indicator=sent[0].isupper()
    changed=0
    #adding the sentences into list if no change is supposed to be performed
    if nchange[0]==0:
        if (indicator):
            toadd=linemodtotal.capitalize()
        else:
            toadd=linemodtotal
        noisesent6.append(toadd)
    index=-1
    #the replacement order of the tokens could also be randomized by making the following line uncommented
    #random.shuffle(tochange)
    while(changed<nchange[-1]):
        index+=1
        if(index==len(tochange)):
            break
        word=tochange[index][0]
        #print("Word to change: {}".format(word))
        #if the embedding of the word in process could be found in the vector space:
        try:
            check=embeddings[word]
            similar=similars(embeddings, word,5) #candidates are obtained
            toreplace=similar[0] #most similar one is chosen
            #print("TOREPLACE: {}".format(toreplace))
            #word is replaced with the candidate accompanying "#" characters as boundaries
            #this is done to prevent the replacement of the in-word and multiple occurences of the words in process 
            pattern='(?<![a-zA-Z#])'+word+'(?![a-zA-Z#])' 
            linemodtotal=re.sub(pattern,'#'+toreplace+'#',linemodtotal,1)
            #print(linemodtotal)
            changed+=1  
        except:
            continue
        #if the desired number of words are replaced
        if changed==nchange[0]:
            #indicator serves as a sign of the capitalized sentence
            #the "#" characters are removed at the end
            if (indicator):
                toadd=linemodtotal.replace('#','').capitalize()
            else:
                toadd=linemodtotal.replace('#','')
            #the augmented sentences added to the list defined at the beginning
            noisesent6.append(toadd)
            #print(toadd)
            break

0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [12]:
#the path where the document containing the augmented samples will be saved to
FINALPATH="/content/"

noiselist=[noisesent6]
for element in noiselist:
    folder="0.6" #Could also bi iterated due to percentages. Here we use only one percentage value therefore it is strictly defined 
    final=[]
    start=0
    for i in range(0,len(nsentence)):
        toaddm=''
        n=nsentence[i]
        #merging the sentences that are contained in the same maxim
        for j in range(start,start+n):
            toaddm=toaddm+' '+element[j]
        toaddm=toaddm[1:]
        final.append(toaddm)
        start=start+n 
    #saving the augmented samples into a text document in the given final path
    f=open(FINALPATH+'/maximsamples_gloveaug'+folder[-1]+'0.txt', 'w',encoding="utf8")
    for line in final:
        f.write(line+'\n')
    f.close()